In [1]:
%matplotlib inline

import numpy as np
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, Polygon

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

# The current version of NumPy available from conda is issuing a warning 
# message that some behavior will change in the future when used with the 
# current version of matplotlib available from conda. This cell just keeps
# that warning from being displayed.
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## Create a Color Mapper

We'll need a way of mapping unemployment rates to hot and cold colors, and what we really want is something with a super simple interface, i.e., we just want to call it and pass in a value and get back a color. Unfortunately, we just can't grab a color map from matplotlib and pass in an unemployment rate. The problem with doing this is that a color map expects a float value between 0 and 1 and our data ranges from roughly 2-30. So, every value in our data falls outside of the range of the color map, and so we end up with a map that is just black across all counties. 

Now, a color map also supports passing in integer values, and it maps those values to one of 256 colors. Given this option, we could simply call `int` on each value before passing it into the color map, but unfortunately, this creates a new problem. Since our data consists of values between 2 and 30, all of our values are on the low end of the spectrum when compared to the color map's range of 0-255. With this method, we end up with an extremely "watered down" map where all of the counties look a nearly identical shade of light yellow.

The solution to both of these problems is to normalize each unemployment rate before passing it into our color map. Fortunately, matplotlib provides the `matploltlib.colors.Normalize` class to make this bit extremely easy. But, as I mentioned above, we want a simple interface that allows us to pass in a value and get a color. One way to accomplish this is through a callable object that encapsulates the creation of the normalization function. Again, matplotlib provides what we need to make this really easy. The `pyplot.cm.ScalarMappable` class is a mixin class for adding color map functionality to custom classes. We use this class below to create our `HeatMapper` helper class.

The `HeatMapper` take a single parameter, a list of values, and creates a normalization function and calls 

In [2]:
class HeatMapper(plt.cm.ScalarMappable):
    """A callable that maps cold colors to low values, and hot to high.
    """
    def __init__(self, data=None):
        norm = mpl.colors.Normalize(vmin=min(data), vmax=max(data))
        cmap = plt.cm.hot_r
        super(HeatMapper, self).__init__(norm, cmap)
        
    def __call__(self, value):
        return self.to_rgba(value)